In [7]:
%%writefile search_phonebook.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

struct Contact {
  char id[50];
  char name[50];
  char number[50];
};

__device__ bool check(char* str1, char* str2, int len) {
  for(int i = 0; str1[i] != '\0'; i++) {
    int j = 0;
    while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
      j++;
    }
    if(j == len-1) {
      return true;
    }
  }
  return false;
}

__global__ void searchPhonebook(Contact* phonebook, int num_contacts, char* search_name, int name_length) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if(idx < num_contacts) {
    if(check(phonebook[idx].name, search_name, name_length)) {
      printf("%s %s %s\n", phonebook[idx].id, phonebook[idx].name, phonebook[idx].number);
    }
  }
}

int main(int argc, char* argv[]) {
  if(argc != 3) {
    cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
    return 1;
  }

  string search_name = argv[1];
  int num_threads = atoi(argv[2]);
  // Mount Google Drive and copy the location
  string file_name = "/content/sample_data/phonebook1.txt";
  //string file_name = "phonebook1.txt";

  vector<Contact> phonebook;

  ifstream file(file_name);
  if(!file.is_open()) {
    cerr << "Error opening file: " << file_name << endl;
    return 1;
  }
  else {
    Contact contact;
    string line;
    while(getline(file, line)) {
      /* Format: "id","name","phone_number"
      int pos = line.find(",");
      strcpy(contact.id, line.substr(1, pos-2).c_str());
      line = line.substr(pos+1);
      pos = line.find(",");
      strcpy(contact.name, line.substr(1, pos-2).c_str());
      strcpy(contact.number, line.substr(pos+2, line.size()-pos-4).c_str());
      phonebook.push_back(contact);
      */

      // Format: "name","phone_number"
      int pos = line.find(",");
      // Extract name (without the quotes)
      strcpy(contact.name, line.substr(1, pos - 2).c_str());

      // Extract number (also without quotes)
      strcpy(contact.number, line.substr(pos + 2, line.size() - pos - 4).c_str());

      phonebook.push_back(contact);
    }
    file.close();
  }
  int num_contacts = phonebook.size();
  Contact* device_phonebook;
  cudaMalloc((void**)&device_phonebook, sizeof(Contact)*num_contacts);
  cudaMemcpy(device_phonebook, phonebook.data(), sizeof(Contact)*num_contacts, cudaMemcpyHostToDevice);

  int name_length = search_name.length() + 1;
  char* device_search_name;
  cudaMalloc((void**)&device_search_name, name_length);
  cudaMemcpy(device_search_name, search_name.c_str(), name_length, cudaMemcpyHostToDevice);

  for(int i = 0; i < num_contacts; i += num_threads) {
    int thread_count = min(num_contacts-i, num_threads);
    searchPhonebook<<<1, thread_count>>>(device_phonebook + i, thread_count, device_search_name, name_length);
    cudaDeviceSynchronize();
  }

  cudaFree(device_phonebook);
  cudaFree(device_search_name);

  return 0;
}

Overwriting search_phonebook.cu


In [13]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook


In [20]:
!pip install pycuda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 11.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2026.1-cp312-cp312-linux_x86_64.whl size=659449 sha256=9138aff2f0a0185f6bffbc768c97e363d39a5dbb0a42b131e20ab3491fcf581c
  Stored in directory: /root/.cache/pip/wheels/90/2a/71/75ec0cc316cc0ff494bfffa2935e02580129cb7f859a0cfd8f
Successfully built pycuda


In [28]:
!time ./search_phonebook SHAKIRA 100 > output1.txt


real	0m0.135s
user	0m0.014s
sys	0m0.113s



real	0m18.743s
user	0m10.888s
sys	0m4.768s
